# Renaming Team Logos to Use in Tableau

In [2]:
import os
import pandas as pd
import unicodedata
import re
from rapidfuzz import process

# === CONFIGURATION ===
SOURCE_DIR = "../../../visuals/logos"  # Root directory with league subfolders
TEAM_CSV = "../../../data/teams/cleaned/all_league_teams.xlsx"  # Your team list file

# === HELPER FUNCTION ===
def normalize_filename(name):
    name = unicodedata.normalize("NFD", name).encode("ascii", "ignore").decode("utf-8")
    name = re.sub(r"[^\w]+", "_", name).strip("_")
    return name + ".png"

# === STEP 1: Load official team names from Excel ===
df = pd.read_excel(TEAM_CSV)
official_teams = df["Team Name"].dropna().unique().tolist()

# Build official filename map
normalized_map = {normalize_filename(name).lower(): name for name in official_teams}
official_files = {name: normalize_filename(name) for name in official_teams}

# === Manual Rename Dictionary ===
manual_renames = {
    "Mainz_05.png": "1_FSV_Mainz_05.png",
    "St_Pauli.png": "FC_St_Pauli.png",
    "Union_Berlin.png": "1_FC_Union_Berlin.png",
    "Werder_Bremen.png": "SV_Werder_Bremen.png",
    "Celta_Vigo.png": "Celta_de_Vigo.png",
    "Las_Palmas.png": "UD_Las_Palmas.png",
    "Le_Havre.png": "Le_Havre_AC.png",
    "Rennes.png": "Stade_Rennais_FC.png",
    "Saint-Étienne.png": "AS_Saint_Etienne.png"
}

# === STEP 2: Traverse files and rename ===
renamed_files = []

for league_folder in os.listdir(SOURCE_DIR):
    league_path = os.path.join(SOURCE_DIR, league_folder)
    if not os.path.isdir(league_path):
        continue

    for file in os.listdir(league_path):
        if not file.lower().endswith(".png"):
            continue

        file_path = os.path.join(league_path, file)
        name_without_ext = os.path.splitext(file)[0]

        # --- Try fuzzy match ---
        best_match, score, _ = process.extractOne(name_without_ext, official_teams)

        if score >= 85:
            new_filename = official_files[best_match]
            new_path = os.path.join(league_path, new_filename)

            if file_path != new_path:
                os.rename(file_path, new_path)
                renamed_files.append((file, new_filename, league_folder))
                print(f"✅ Renamed: {file} → {new_filename}")

        else:
            # --- Try manual fallback ---
            if file in manual_renames:
                new_filename = manual_renames[file]
                new_path = os.path.join(league_path, new_filename)

                if file_path != new_path:
                    os.rename(file_path, new_path)
                    renamed_files.append((file, new_filename, league_folder))
                    print(f"🛠️ Manually Renamed: {file} → {new_filename}")
            else:
                print(f"❌ No good match for: {file}")

🛠️ Manually Renamed: Mainz_05.png → 1_FSV_Mainz_05.png
🛠️ Manually Renamed: St_Pauli.png → FC_St_Pauli.png
🛠️ Manually Renamed: Union_Berlin.png → 1_FC_Union_Berlin.png
🛠️ Manually Renamed: Werder_Bremen.png → SV_Werder_Bremen.png
🛠️ Manually Renamed: Celta_Vigo.png → Celta_de_Vigo.png
🛠️ Manually Renamed: Las_Palmas.png → UD_Las_Palmas.png
🛠️ Manually Renamed: Le_Havre.png → Le_Havre_AC.png
🛠️ Manually Renamed: Rennes.png → Stade_Rennais_FC.png
🛠️ Manually Renamed: Saint-Étienne.png → AS_Saint_Etienne.png
